In [2]:
!pip install tensorflow
!pip install tf2onnx
!pip install numpy

In [ ]:
!pip install onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00


In [3]:
import os
import sys
import time
import argparse
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Для экспорта в ONNX и проверки модели
import tf2onnx
import onnx
import onnxruntime as ort

2025-04-03 11:50:59.408678: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 11:50:59.705964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743655859.811631   47831 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743655859.843008   47831 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743655860.083020   47831 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
# Парсинг аргументов командной строки
# parser = argparse.ArgumentParser()
# parser.add_argument('data_dir', type=str, help="Путь для хранения/загрузки данных (будет использован для сохранения модели)")
# parser.add_argument('class_num', type=int, help="Количество классов (2, 10 или 20)")
# parser.add_argument('train_round', type=int, help="Количество эпох для обучения")
# args = parser.parse_args()

# DATA_DIR = args.data_dir
DATA_DIR = "FlowAllLayers/"
# CLASS_NUM = args.class_num
CLASS_NUM = 10
# TRAIN_ROUND = args.train_round
TRAIN_ROUND = 1

# Словари для наименования классов (пример – можно расширить по необходимости)
dict_2class = {0: 'Benign', 1: 'Malware'}
dict_10class_benign = {0: 'BitTorrent', 1: 'Facetime', 2: 'FTP', 3: 'Gmail', 4: 'MySQL', 5: 'Outlook', 6: 'Skype', 7: 'SMB', 8: 'Weibo', 9: 'WorldOfWarcraft'}
dict_10class_malware = {0: 'Cridex', 1: 'Geodo', 2: 'Htbot', 3: 'Miuref', 4: 'Neris', 5: 'Nsis-ay', 6: 'Shifu', 7: 'Tinba', 8: 'Virut', 9: 'Zeus'}
dict_20class = {**dict_10class_benign, **dict_10class_malware}  # объединение двух словарей

# Загрузка MNIST через tf.keras (данные автоматически загружаются и кэшируются)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Приведение данных к формату float и нормализация [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Добавляем размерность канала (MNIST – чёрно-белые изображения)
x_train = np.expand_dims(x_train, -1)  # форма (num_samples, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)

# Если модель предназначена для классов, отличных от стандартных 10,
# можно добавить маппинг или использовать только подмножество данных.
# Здесь предполагается, что для CLASS_NUM==10 используется весь MNIST.
if CLASS_NUM not in [2, 10, 20]:
    print("Не поддерживаемое количество классов. Используйте 2, 10 или 20.")
    sys.exit(1)

# Преобразование меток в one-hot представление (учтите, что для 2 и 20 классов нужна соответствующая предобработка)
y_train = keras.utils.to_categorical(y_train, 10)
y_test  = keras.utils.to_categorical(y_test, 10)

# Если требуется изменить число классов (например, для двоичной классификации),
# можно взять только часть классов или объединить их. Здесь оставляем стандартное число классов MNIST.
# В дальнейшем при выводе результатов можно использовать соответствующий словарь:
if CLASS_NUM == 2:
    class_dict = dict_2class
elif CLASS_NUM == 10:
    # Для демонстрации можно выбрать benign или malware по имени каталога DATA_DIR
    folder = os.path.basename(DATA_DIR)
    if folder.startswith('Benign'):
        class_dict = dict_10class_benign
    else:
        class_dict = dict_10class_malware
elif CLASS_NUM == 20:
    class_dict = dict_20class

# Определяем модель с использованием Keras API
inputs = keras.Input(shape=(28, 28, 1), name="input")
x = layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = layers.Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax', name="output")(x)  # обучаем на MNIST (10 классов)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(optimizer=keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Формирование имени модели и пути сохранения
model_dir = f"model_{CLASS_NUM}class"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
checkpoint_path = os.path.join(model_dir, f"{model_dir}.weights.h5")

# Если сохранённая модель существует – загружаем веса, иначе обучаем
if os.path.exists(checkpoint_path + ".index"):
    model.load_weights(checkpoint_path)
    print("Model restored:", checkpoint_path)
else:
    model.fit(x_train, y_train,
              batch_size=50,
              epochs=TRAIN_ROUND,
              validation_split=0.1)
    model.save_weights(checkpoint_path)
    print("Model saved in file:", checkpoint_path)

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 32)          │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 14, 14, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       3,212,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,274,634 (12.49 MB)

 Trainable params: 3,274,634 (12.49 MB)

 Non-trainable params: 0 (0.00 B)

1080/1080 ━━━━━━━━━━━━━━━━━━━━ 125s 114ms/step - accuracy: 0.9051 - loss: 0.2928 - val_accuracy: 0.9890 - val_loss: 0.0389
Model saved in file: model_10class/model_10class.weights.h5


In [ ]:
# Оценка точности на тестовом наборе
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: {:.4f}".format(test_acc))

Test accuracy: 0.9873


In [ ]:
# Экспорт модели в формат ONNX
# Для экспорта задаём сигнатуру входного тензора
spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
output_path = os.path.join(model_dir, f"{model_dir}.onnx")
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
print("ONNX model exported to:", output_path)

ONNX model exported to: model_10class/model_10class.onnx


In [ ]:
# Проверка корректности экспортированной ONNX модели
onnx_model = onnx.load(output_path)
onnx.checker.check_model(onnx_model)
print("ONNX model check passed.")

ONNX model check passed.


In [ ]:
# Пробный запуск инференса через onnxruntime для проверки работы модели
ort_session = ort.InferenceSession(output_path)
def run_onnx_inference(input_data):
    # onnxruntime требует numpy-массив
    ort_inputs = {ort_session.get_inputs()[0].name: input_data}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs[0]

# Пример: получение предсказания для одного изображения из тестового набора
sample = x_test[:1]
onnx_pred = run_onnx_inference(sample)
print("ONNX model prediction (first sample):", onnx_pred)

ONNX model prediction (first sample): [[1.6238681e-11 1.1650045e-10 1.2089139e-07 8.3929422e-07 2.0441404e-13
  3.7070047e-10 7.7068495e-15 9.9999905e-01 2.2729889e-11 2.6208479e-08]]


In [ ]:
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

# Загрузка MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Нормализация
x_test = x_test.astype("float32") / 255.0

# Выбор первого тестового изображения
sample = x_test[0]  # Размер: 28×28

# Сохранение как BMP
plt.imsave("test_image.bmp", sample, cmap="gray")
print("Тестовое изображение сохранено как test_image.bmp")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Тестовое изображение сохранено как test_image.bmp


In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(_, _), (x_test, _) = mnist.load_data()

# Приведение к формату float и нормализация
x_test = x_test.astype('float32') / 255.0

# Добавляем размерность канала: (num_samples, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)

# Выбираем один образец (например, первый)
sample = x_test[0]  # shape: (28, 28, 1)

# Если требуется пакетный ввод, можно добавить размерность batch:
sample = np.expand_dims(sample, 0)  # теперь shape: (1, 28, 28, 1)

# Для записи в бинарный файл нам нужен одномерный массив из float32
# (Ожидается, что C++ код считает 1*28*28*1 элементов)
sample_flat = sample.reshape(-1)

# Сохраняем в файл (будет создан файл 'prepared_input' размером 28*28*1*4 байт)
sample_flat.tofile("prepared_input")
print("Входной тензор сохранён в файле 'prepared_input'")


Входной тензор сохранён в файле 'prepared_input'
